In [1]:
import numpy as np
movie_data = np.load('movie_data_v1.npy', allow_pickle=True)[()]

In [2]:
import random
random.seed(1230)

movie_ids = list(movie_data.keys())
num_movie = len(movie_ids)
random.shuffle(movie_ids)

split_1, split_2 = int(num_movie * 0.85), int(num_movie * 0.9)
train_ids = movie_ids[:split_1]
valid_ids = movie_ids[split_1:split_2]
test_ids = movie_ids[split_2:]
print(len(train_ids), len(valid_ids), len(test_ids))

524 31 62


In [3]:
genres = set()
for movie_id in movie_ids:
    cur_genres = eval(movie_data[movie_id]['meta'][2])
    for genre in cur_genres:
        genres.add(genre)
genre_map = dict(zip(genres, list(range(len(genres)))))
print(len(genres))
np.save('genre_map.npy', genre_map)

24


In [5]:
vocab = np.load('vocab.npy')
vocab_map = dict(zip(vocab, list(range(1, len(vocab)+1))))
np.save('vocab_map.npy', vocab_map)

In [6]:
def generate_data(name, ids):
    emb_data, token_data, ratings, genres = [], [], [], []
    for movie_id in ids:
        cur_emb = []
        cur_tokens = []
        for conversation in movie_data[movie_id]['conversation']:
            for lid, cid, tokens, emb in conversation:
                cur_emb.append((lid, emb))
                cur_tokens.append((lid, [vocab_map[x] for x in tokens]))
        cur_emb.sort(key=lambda x:x[0])
        cur_tokens.sort(key=lambda x:x[0])
        cur_emb = [x[1] for x in cur_emb]
        cur_tokens = [x[1] for x in cur_tokens]
        emb_data.append(cur_emb)
        token_data.append(cur_tokens)
        ratings.append(movie_data[movie_id]['meta'][1])
        genres.append([genre_map[x] for x in eval(movie_data[movie_id]['meta'][2])])
    np.savez(name + '_pretrain.npz', data=emb_data, ratings=ratings, genres=genres)
    np.savez(name + '_vocab.npz', data=token_data, ratings=ratings, genres=genres)

In [7]:
# generate_data('movie', movie_ids)

small_ids = random.sample(movie_ids, 100)
generate_data('movie_test', small_ids)

In [8]:
movie_pretrain = np.load('movie_test_pretrain.npz', allow_pickle=True)
movie_vocab = np.load('movie_test_vocab.npz', allow_pickle=True)

In [9]:
len(movie_vocab['data'][0])

832